In [3]:
import numpy as np

In [4]:
#problem 1
class simplexMethod(object):
    def __init__(self, c, A, b):
        if (b < 0).any():
            raise ValueError ("Problem not feasible at origin!")
        self.c = c
        self.A = A
        self.b = b
        
        
        
        
        

In [8]:
simp = simplexMethod
A = np.array([[1, -1], [3, 1], [4, 3]]).reshape(3, 2)
b = np.array([[2, 5, -7]])
c = np.array([[3, 2]]).reshape(2, 1)
simp(c, A, b)

ValueError: Problem not feasible at origin!

In [7]:
if (simp.b < 0).any():
            raise ValueError ("Problem not feasible at origin!")

ValueError: Problem not feasible at origin!